# Importing Libraries

In [1]:
import pandas as pd
from yelpapi import YelpAPI
import os, time, math, json
from tqdm.notebook import tqdm_notebook

This notebook is going to extract data from the Yelp API. Specifically the data for businesses in Miami FL that serve Cuban food. I will start by bringing in my credentials and instantiating the API

# Api

In [2]:
# Opening my API creds
with open('/Users/Ray/.secret/yelp_api.json', 'r') as f:
    login = json.load(f)
    
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiating the API
yelpcall = YelpAPI(login['api-key'], timeout_s = 5.0)
yelpcall

In [4]:
# Setting the keywords for the API calls
location = 'Miami, FL'
term = 'Cuban'

# File 
Now to create a folder and a file to store the results of my api calls

In [5]:
# File name to use later on
json_file = 'Data/previous_results.json'

# IF statement to create the file
file_exists = os.path.isfile(json_file)
if file_exists == False:
    
    # this will create a folder for the json file IF the json file name had a folder
    folder = os.path.dirname(json_file)
    
    if len(folder)>0:
        os.makedirs(folder, exist_ok = True)
        
    # Print a status to the notebook    
    print(f'[info] {json_file} not found. Saving empty list to file')
    
    # Saving an empty list to json_file
    with open(json_file, "w") as f:
        json.dump(obj = [], fp = f)

else:
    print(f'[info] {json_file} already exists')


[info] Data/previous_results.json not found. Saving empty list to file


# Prepping to loop
Next step is going to be to calculate the pagination batch sizes and loop through the available data

In [6]:
with open(json_file, 'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'{n_results} previous results were found in JSON file')

0 previous results were found in JSON file


In [7]:
# running the first call to get the total results and the total per page
results = yelpcall.search_query(location = location, term = term)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [8]:
# Total
total_results = results['total']

# Total per page
total_per_page = len(results['businesses'])

# Number of loops needed
n_loops = math.ceil((total_results - n_results)/total_per_page)

print(f'{total_results} total results')
print(f'{total_per_page} total results per page')
print(f'{n_loops} loops needed')

962 total results
20 total results per page
49 loops needed


# Constructing the loop
Now to create the loop, but first lets add the first results to the json file

In [9]:
# Extending previous results
previous_results.extend(results['businesses'])

# Saving
with open(json_file, 'w') as f:
    json.dump(previous_results, f)

In [11]:
# Using tqdm to create a progress bar
for i in tqdm_notebook(range(1, n_loops + 1)):
    # Retrieving the previous results
    with open(json_file, 'r') as f:
        previous_results = json.load(f)
        
    # Calculating the length of previous results to use as an offset
    n_results = len(previous_results)
    
    # API call, retrieving a JSON string
    results = yelpcall.search_query(location = location, term = term, offset = n_results)
    
    # Extending previous results
    previous_results.extend(results['businesses'])
    
    # Saving
    with open(json_file, 'w') as f:
        json.dump(obj = previous_results, fp = f)
        
    # 200 ms Pause
    time.sleep(.2)
    

  0%|          | 0/49 [00:00<?, ?it/s]

# Converting to dataframe

In [12]:
df = pd.read_json(json_file)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,UXHxLN3DcDGI57uDIfCuJA,olds-havana-cuban-bar-and-cocina-miami,Old's Havana Cuban Bar & Cocina,https://s3-media4.fl.yelpcdn.com/bphoto/OyMD-x...,False,https://www.yelp.com/biz/olds-havana-cuban-bar...,2161,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.5,"{'latitude': 25.7655942148975, 'longitude': -8...","[pickup, delivery]",$$,"{'address1': '1442 SW 8th St', 'address2': '',...",+17865182196,(786) 518-2196,2093.858301
1,hZm7TunlrksQbgS0ssXbUg,versailles-miami-4,Versailles,https://s3-media1.fl.yelpcdn.com/bphoto/VaSpiR...,False,https://www.yelp.com/biz/versailles-miami-4?ad...,6151,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.0,"{'latitude': 25.765039080853928, 'longitude': ...",[],$$,"{'address1': '3555 SW 8th St', 'address2': '',...",+13054440240,(305) 444-0240,3127.739196
2,zGf5kkYEU01NQ04DRYVPNQ,doce-provisions-miami,Doce Provisions,https://s3-media3.fl.yelpcdn.com/bphoto/spQ-Zr...,False,https://www.yelp.com/biz/doce-provisions-miami...,929,"[{'alias': 'gastropubs', 'title': 'Gastropubs'...",4.5,"{'latitude': 25.767862, 'longitude': -80.214191}","[pickup, delivery]",$$,"{'address1': '541 SW 12th Ave', 'address2': No...",+17864520161,(786) 452-0161,2162.106175
3,nTsEr_CQqsA8zwLV4kT5nA,cafe-la-trova-miami,Cafe La Trova,https://s3-media3.fl.yelpcdn.com/bphoto/CJenu6...,False,https://www.yelp.com/biz/cafe-la-trova-miami?a...,891,"[{'alias': 'cocktailbars', 'title': 'Cocktail ...",4.5,"{'latitude': 25.7662659, 'longitude': -80.21063}","[pickup, delivery]",$$,"{'address1': '971 SW 8th St', 'address2': '', ...",+17866154379,(786) 615-4379,2534.517384
4,gmDna9f57vWTlSrdrEuVEg,sanguich-de-miami-miami-2,Sanguich De Miami,https://s3-media2.fl.yelpcdn.com/bphoto/PUj9xl...,False,https://www.yelp.com/biz/sanguich-de-miami-mia...,1034,"[{'alias': 'cuban', 'title': 'Cuban'}, {'alias...",4.5,"{'latitude': 25.7656049, 'longitude': -80.2285...",[delivery],$$,"{'address1': '2057 SW 8th St', 'address2': '',...",+13055390969,(305) 539-0969,1859.388202
